In [1]:
%pylab inline
import nltk
from nltk.corpus import words

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Grab all english words.
w = words.words()

In [3]:
#Sort each word's characters in lowercase; store in sequence.
w_proc = [''.join(sorted(s.lower())) for s in w]

In [4]:
#Map each processed word to every word that shares its characters.
w_map = {}
for i, s in enumerate(w_proc):
    word = w[i]
    if w_map.get(s):
        w_map[s].append(word)
    else:
        w_map[s] = [word]

In [5]:
import itertools as it
#Get all possible character combinations for a given string of all lengths.
def get_combs(s):
    res = [[''.join(sorted(c)).strip() for c in it.combinations(s,i)] for i in range(len(s))]
    combs = []
    for l in res:
        combs += l
    combs = set(combs)
    return combs

In [6]:
#Find matching words that can be constructed from these combs.
def get_buildable(s):

    c = get_combs(s)
    buildable = []
    for x in c:
        match = w_map.get(x)
        if match:
            buildable += [s.lower() for s in match]
    
    return set(buildable)

In [7]:
#Parallelize it!
import pandas as pd
# Import
from pandarallel import pandarallel

# Initialization
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data tranfer (pipe) to transfer data between the main process and workers.


In [8]:
'''
TODO: run this
NOTE: Takes VERY long time to run - save results!
'''
#Run the above function for ALL words.
bfreqs = pd.Series(w).parallel_apply(lambda x: len(get_buildable(x)))

In [10]:
bfreqs.to_csv('bfreqs')

/Users/mgbvox/anaconda3/envs/data_science/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [23]:
bfreqs.sort_values(ascending=False)/len(w)

158764    0.137102
23601     0.093535
158763    0.092867
187326    0.091292
158627    0.080904
            ...   
226497    0.000000
226498    0.000000
234937    0.000000
67699     0.000000
0         0.000000
Length: 236736, dtype: float64

In [12]:
w[158764]

'pseudolamellibranchiate'